In [1]:
import sqlite3
import pandas as pd
import logging
from datetime import datetime
import os
import json

# Part 1 : read files and store in JSON file

In [2]:

# Configuration for folders and JSON file path
FOLDERS = ["BRONZE", "SILVER", "GOLD"]
JSON_FILE_PATH = "sqllite_config.json"

# --- 1. Fetch Filenames ---
csv_files = {}

for folder in FOLDERS:
    try:
        # List all files ending with .csv in the folder
        files = [f for f in os.listdir(folder) if f.endswith(".csv")]
        csv_files[folder] = files
    except FileNotFoundError:
        print(f"Warning: Folder '{folder}' not found. Skipping file fetching for this folder.")
        csv_files[folder] = []


# --- 2. Prepare 'files' dictionary for JSON update ---
file_config = {}

# Bronze: Remains single file (first file in the list)
if csv_files["BRONZE"]:
    file_config["bronze_csv_file"] = csv_files["BRONZE"][0]

# Silver: Updated to include ALL files found as a list
if csv_files["SILVER"]:
    file_config["silver_csv_files_list"] = csv_files["SILVER"]
else:
    file_config["silver_csv_files_list"] = [] # Ensure the key exists even if empty

# CORRECTION: Gold is now updated to include ALL files found as a list
if csv_files["GOLD"]:
    file_config["gold_csv_files_list"] = csv_files["GOLD"]
else:
    file_config["gold_csv_files_list"] = [] # Ensure the key exists even if empty


# --- 3. Update JSON File with Filenames ---
try:
    # Read the existing data
    with open(JSON_FILE_PATH, 'r') as f:
        config_data = json.load(f)
except (FileNotFoundError, json.JSONDecodeError):
    # Initialize if file doesn't exist or is invalid
    config_data = {}

# Update the 'files' section (keeping other data intact)
# Use .update() to merge the new file configurations into the existing 'files' dictionary
existing_files = config_data.get("files", {})
existing_files.update(file_config)
config_data["files"] = existing_files


# Write the modified data back to the file
print(f"Writing updated filenames to {JSON_FILE_PATH}...")
with open(JSON_FILE_PATH, 'w') as f:
    json.dump(config_data, f, indent=4)

print(f"Part 1: Filenames updated in {JSON_FILE_PATH}.")

Writing updated filenames to sqllite_config.json...
Part 1: Filenames updated in sqllite_config.json.


# Read filenames and create table names dynamically

In [3]:
JSON_FILE_PATH = "sqllite_config.json"
TIER_KEYWORDS = ["bronze", "silver", "gold"]

# Function to implement the simplified table naming logic (remains unchanged)
def derive_custom_table_name(filename):
    """
    Extracts the table name starting from the tier keyword and includes
    the next two underscore-separated words, effectively stopping after the 3rd word.
    Example: 'ailab_bronze_eres_flight_data1.csv' -> 'bronze_eres_flight'
    """
    if not filename:
        return None

    # 1. Remove the extension and convert to lowercase for searching
    base_name, _ = os.path.splitext(filename)
    base_name_lower = base_name.lower()
    
    # 2. Find the starting position of the tier keyword
    start_index = -1
    
    for tier in TIER_KEYWORDS:
        if tier in base_name_lower:
            start_index = base_name_lower.find(tier)
            break
    
    if start_index != -1:
        # Start the relevant part of the name from the tier keyword
        custom_name = base_name[start_index:]
        
        # Split by underscore
        parts = custom_name.split('_')
        
        # We take the tier word (parts[0]) + the next two words (parts[1] and parts[2])
        final_parts = parts[:3]
        
        if final_parts:
            # Join the first three parts (e.g., "bronze_eres_flight")
            return "_".join(final_parts)
        
        return custom_name 
            
    # Fallback if no tier keyword found
    return base_name

# --- 1. Read JSON file to get filenames ---
try:
    print(f"Reading existing configuration from {JSON_FILE_PATH}...")
    with open(JSON_FILE_PATH, 'r') as f:
        config_data = json.load(f)
except (FileNotFoundError, json.JSONDecodeError):
    print(f"Error: Could not read or decode JSON file at {JSON_FILE_PATH}. Aborting Part 2.")
    exit()

# Safely extract the filenames dictionary
file_config = config_data.get("files", {})

# --- MODIFIED: Get single file for Bronze, and lists for Silver/Gold ---
bronze_file = file_config.get("bronze_csv_file")
silver_files_list = file_config.get("silver_csv_files_list", []) # Default to empty list
gold_files_list = file_config.get("gold_csv_files_list", [])     # Default to empty list


# --- 2. Prepare the NEW Nested Table Dictionaries ---

# Initialize the dictionaries for the new nested structures
bronze_tables = {}
silver_tables = {}
gold_tables = {}
all_generated_table_names = []


# Helper function to process files and populate the respective table dictionary
def process_files_for_tier(file_list_or_single_file, tier_key, table_dict):
    # Ensure we are working with an iterable list, even if it's a single string
    if isinstance(file_list_or_single_file, str):
        files_to_process = [file_list_or_single_file]
    elif file_list_or_single_file is None:
        files_to_process = []
    else:
        files_to_process = file_list_or_single_file
    
    tier_generated_names = []
    
    for file_name in files_to_process:
        if file_name:
            # Derive the table name using the custom logic
            table_name = derive_custom_table_name(file_name)
            
            # Enforce desired casing for silver (e.g., 'silver_eres_flight')
            if tier_key == "silver" and table_name.lower().startswith("silver"):
                table_name = "silver" + table_name[6:]
            
            # Use the full filename (with extension removed) for the database file name
            # NOTE: For simplicity, we use the derived table_name + '.db' as the value
            db_file_name = f"{table_name}.db"
            
            # Populate the table dictionary: Key = Table Name, Value = DB File Name
            table_dict[table_name] = db_file_name
            tier_generated_names.append(table_name)
            
    return tier_generated_names


# Generate configurations for all three tiers
bronze_names = process_files_for_tier(bronze_file, "bronze", bronze_tables)
silver_names = process_files_for_tier(silver_files_list, "silver", silver_tables)
gold_names = process_files_for_tier(gold_files_list, "gold", gold_tables)


# --- 3. Update JSON File with New Nested Tables ---

# Update the root config_data with the new bronze/silver/gold_tables dictionaries
config_data["bronze_tables"] = bronze_tables
config_data["silver_tables"] = silver_tables
config_data["gold_tables"] = gold_tables

# Remove the old generic 'tables' key if it exists (cleanup from prior attempts)
if "tables" in config_data:
    del config_data["tables"]
    
# Write the modified data back to the file
print(f"Writing updated nested table configuration back to {JSON_FILE_PATH}...")
with open(JSON_FILE_PATH, 'w') as f:
    json.dump(config_data, f, indent=4)


# --- Print Summary ---
print(f"\nPart 2: Configuration file updated successfully.")
print(f" - Bronze Table(s) Generated: **{', '.join(bronze_names) if bronze_names else 'N/A'}**")
print(f" - Silver Table(s) Generated: **{', '.join(silver_names) if silver_names else 'N/A'}**")
print(f" - Gold Table(s) Generated: **{', '.join(gold_names) if gold_names else 'N/A'}**")

Reading existing configuration from sqllite_config.json...
Writing updated nested table configuration back to sqllite_config.json...

Part 2: Configuration file updated successfully.
 - Bronze Table(s) Generated: **bronze_eres_flight**
 - Silver Table(s) Generated: **Ailab_curated_flight, silver_eres_airportcode, silver_eres_flight, silver_eres_seasoncode**
 - Gold Table(s) Generated: **AILab_Consumption_Customer_traveller, Gold_dimension_flightdata**


# load into SQLLITE

In [5]:
import pandas as pd
import sqlite3
import json
import os
import logging
import sys 

# --- Configuration File Path ---
CONFIG_FILE_PATH = "sqllite_config.json"

def load_data_from_csv_to_db():
    """Reads configuration, sets up logging, and loads specified CSV files into their respective SQLite databases."""
    
    # 1. Load Configuration from JSON
    try:
        with open(CONFIG_FILE_PATH, 'r') as f:
            config = json.load(f)
    except Exception as e:
        print(f"❌ FATAL ERROR: Could not load configuration file {CONFIG_FILE_PATH}. Aborting. Error: {e}")
        return

    # --- 2. Setup Logging ---
    
    try:
        log_file = config['logging']['log_file']
    except KeyError:
        log_file = "default_db_operations.log"
        print(f"⚠️ Warning: 'logging' key not found in config. Using default log file: {log_file}")
    
    # Ensure logging is only configured once
    if not logging.getLogger('').handlers:
        logging.basicConfig(
            filename=log_file,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S',
            filemode='a'
        )
        
        console = logging.StreamHandler(sys.stdout)
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(message)s')
        console.setFormatter(formatter)
        logging.getLogger('').addHandler(console)
    
    logging.info(f"--- Data Loading Process Started ---")
    logging.info(f"✅ Configuration loaded successfully from {CONFIG_FILE_PATH}.")


    # --- 3. Extract Configuration Values ---
    
    # Files
    bronze_filename = config['files'].get('bronze_csv_file')
    silver_files = config['files'].get('silver_csv_files_list', [])
    gold_files = config['files'].get('gold_csv_files_list', [])
    
    # Databases
    db_map = {
        "bronze": config['databases']['bronze_database_file'],
        "silver": config['databases']['silver_database_file'],
        "gold": config['databases']['gold_database_file'],
    }
    
    # Schemas (for dynamic mapping)
    silver_schema = config.get('silver_schema', {})
    gold_schema = config.get('gold_schema', {})
    
    # Tables (The actual table name inside the DB)
    try:
        bronze_table_name = list(config['bronze_tables'].keys())[0]
    except (IndexError, KeyError):
        bronze_table_name = None


    # ⬇️ --- 4. Schema Mapping and Processing Function (CORRECTED) --- ⬇️
    
    def get_target_schema_name(tier_key, filename, silver_schema, gold_schema):
        """Determines the target table name based on the 'curated' keyword, 
        using values from the respective schema dictionary."""
        
        # Determine the correct schema dictionary based on the tier
        schema_dict = silver_schema if tier_key == "silver" else gold_schema
        
        # Keyword used for conditional routing in both file and schema value
        CURATION_KEYWORD = "curated" 
        
        # Rule 1: If filename contains "curated" (case-insensitive)
        if CURATION_KEYWORD in filename.lower():
            # Find the value in the schema dictionary that contains "curated"
            for value in schema_dict.values():
                if CURATION_KEYWORD in value:
                    # ✅ Correctly returns 'silver_curated_flight' or 'gold_curated_flight'
                    return value 
            
            logging.warning(f"⚠️ Curation file '{filename}' found, but no matching '{CURATION_KEYWORD}' schema value in {tier_key}_schema.")
            
        # Rule 2: Use the non-curation schema as the default
        else:
            # Find the value that does NOT contain "curated"
            for value in schema_dict.values():
                 if CURATION_KEYWORD not in value:
                     # ✅ Correctly returns 'silver_flight' or 'gold_flight'
                     return value 
                     
            logging.warning(f"⚠️ No default schema found (no schema value without '{CURATION_KEYWORD}') for file '{filename}'.")
            
        return None # Return None if mapping fails


    def process_data_load(tier_name, folder, csv_file, db_file, table_name):
        """Loads a single CSV file into a specified SQLite table."""
        full_csv_path = os.path.join(folder, csv_file)
        
        logging.info(f"\n--- Processing {tier_name.upper()} File: {csv_file} ---")
        logging.info(f"Source Path: {full_csv_path}")
        logging.info(f"Target DB: {db_file}")
        logging.info(f"Target Table: {table_name}")
        
        if not os.path.exists(full_csv_path):
            logging.error(f"❌ ERROR: CSV file not found at {full_csv_path}. Skipping load.")
            return

        try:
            df = pd.read_csv(full_csv_path)
            logging.info(f"   Successfully read {len(df)} rows from CSV.")
            
            conn = sqlite3.connect(db_file)
            
            # The table name is the schema value derived above
            df.to_sql(table_name, conn, if_exists='replace', index=False)

            conn.commit()
            conn.close()
            logging.info(f"   ✅ Data successfully loaded into table '{table_name}' in {db_file}.")

        except Exception as e:
            logging.error(f"❌ ERROR during {tier_name} data loading for {csv_file}: {e}")

    # --- 5. Execute Data Loading ---
    
    # A. Load Bronze Data (Single File)
    logging.info("\n--- EXECUTING BRONZE LOAD ---")
    if bronze_filename and bronze_table_name:
        process_data_load(
            tier_name="bronze",
            folder="BRONZE",
            csv_file=bronze_filename,
            db_file=db_map["bronze"],
            table_name=bronze_table_name
        )

    # B. Load Silver Data (Multiple Files with Schema Mapping)
    logging.info("\n--- EXECUTING SILVER LOAD (Multiple Files) ---")
    for silver_file in silver_files:
        target_table = get_target_schema_name("silver", silver_file, silver_schema, gold_schema)
        
        if target_table:
            process_data_load(
                tier_name="silver",
                folder="SILVER",
                csv_file=silver_file,
                db_file=db_map["silver"],
                table_name=target_table
            )

    # C. Load Gold Data (Multiple Files with Schema Mapping)
    logging.info("\n--- EXECUTING GOLD LOAD (Multiple Files) ---")
    for gold_file in gold_files:
        # Note: We pass silver_schema and gold_schema, but only the appropriate one is used inside the function
        target_table = get_target_schema_name("gold", gold_file, silver_schema, gold_schema)
        
        if target_table:
            process_data_load(
                tier_name="gold",
                folder="GOLD",
                csv_file=gold_file,
                db_file=db_map["gold"],
                table_name=target_table
            )
    
    logging.info("\n--- Data Loading Process Finished ---")

if __name__ == "__main__":
    load_data_from_csv_to_db()

--- Data Loading Process Started ---
✅ Configuration loaded successfully from sqllite_config.json.

--- EXECUTING BRONZE LOAD ---

--- Processing BRONZE File: ailab_bronze_eres_flight_data1.csv ---
Source Path: BRONZE\ailab_bronze_eres_flight_data1.csv
Target DB: bronze_data.db
Target Table: bronze_eres_flight
   Successfully read 46 rows from CSV.
   ✅ Data successfully loaded into table 'bronze_eres_flight' in bronze_data.db.

--- EXECUTING SILVER LOAD (Multiple Files) ---

--- Processing SILVER File: Ailab_curated_flight.csv ---
Source Path: SILVER\Ailab_curated_flight.csv
Target DB: silver_data.db
Target Table: silver_curated_flight
   Successfully read 50 rows from CSV.
   ✅ Data successfully loaded into table 'silver_curated_flight' in silver_data.db.

--- Processing SILVER File: ailab_silver_eres_airportcode.csv ---
Source Path: SILVER\ailab_silver_eres_airportcode.csv
Target DB: silver_data.db
Target Table: silver_flight
   Successfully read 100 rows from CSV.
   ✅ Data success